# Model recommendation with lighfm

### Import libraries

In [1]:
import os
import sys
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import scipy
import numpy as np
import pandas as pd
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k, recall_at_k
from lightfm.cross_validation import random_train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from lightfm import cross_validation
import scipy.sparse as sp
from scipy import sparse

In [2]:
from lightfm import LightFM
from sklearn.base import clone


class LightFMResizable(LightFM):
    """A LightFM that resizes the model to accomodate new users,
    items, and features"""

    def fit_partial(
        self,
        interactions,
        user_features=None,
        item_features=None,
        sample_weight=None,
        epochs=1,
        num_threads=1,
        verbose=False,
    ):
        try:
            self._check_initialized()
            self._resize(interactions, user_features, item_features)
        except ValueError:
            # This is the first call so just fit without resizing
            pass

        super().fit_partial(
            interactions,
            user_features,
            item_features,
            sample_weight,
            epochs,
            num_threads,
            verbose,
        )

        return self

    def _resize(self, interactions, user_features=None, item_features=None):
        """Resizes the model to accommodate new users/items/features"""

        no_components = self.no_components
        no_user_features, no_item_features = interactions.shape  # default

        if hasattr(user_features, "shape"):
            no_user_features = user_features.shape[-1]
        if hasattr(item_features, "shape"):
            no_item_features = item_features.shape[-1]

        if (
            no_user_features == self.user_embeddings.shape[0]
            and no_item_features == self.item_embeddings.shape[0]
        ):
            return self

        new_model = clone(self)
        new_model._initialize(no_components, no_item_features, no_user_features)

        # update all attributes from self._check_initialized
        for attr in (
            "item_embeddings",
            "item_embedding_gradients",
            "item_embedding_momentum",
            "item_biases",
            "item_bias_gradients",
            "item_bias_momentum",
            "user_embeddings",
            "user_embedding_gradients",
            "user_embedding_momentum",
            "user_biases",
            "user_bias_gradients",
            "user_bias_momentum",
        ):
            # extend attribute matrices with new rows/cols from
            # freshly initialized model with right shape
            old_array = getattr(self, attr)
            old_slice = [slice(None, i) for i in old_array.shape]
            new_array = getattr(new_model, attr)
            new_array[tuple(old_slice)] = old_array
            setattr(self, attr, new_array)

        return self

### Defining variables

In [3]:
import json

with open('config.json', 'r') as f:
    config = json.load(f)

In [4]:
# NUM_ITEMS_TRAIN = 108573
NUM_ITEMS_TRAIN = 63893
# NUM_ITEMS_TRAIN = 231305

NO_EPOCHS = 80

NUM_COMPONENTS = 30

NUM_THREADS = 4

### Retrieve data

In [5]:
dtype_df_valid = {
"userId" : 'string',
"userType" : 'category',
"history" : 'string',
"timestampHistory" : 'string'
}

In [6]:
import pandas as pd
# path config

df_valid = pd.read_csv(config["VALID_DF"],dtype=dtype_df_valid,nrows=10000)
df_valid.head(3)

,userId,userType,history,timestampHistory
0,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,['be89a7da-d9fa-49d4-9fdc-388c27a15bc8' '01c5...,[1660533136590 1660672113513]
1,d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdee...,Logged,['77901133-aee7-4f7b-afc0-652231d76fe9'],[1660556860253]
2,755062dd39a48809880cf363b04268c3af2c003088cde0...,Logged,['857aa90f-a7ec-410d-ba82-dfa4f85d4e71'],[1660561649242]


In [7]:
dtype_df_train_score = {
"userId" : 'string',
"userType" : 'category',
"history" : 'string',
"score" : 'Float32'
}

In [8]:
import pandas as pd

# df_ratings = pd.read_csv(config["DF_TRAIN_SCORES"], dtype=dtype_df_train_score)
df_ratings = pd.read_csv(config["DF_TRAIN_SCORES"], dtype=dtype_df_train_score, nrows=500010)
df_ratings.drop(columns=["Unnamed: 0"],inplace=True)
df_ratings.head(3)

,userId,history,userType,score
0,fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27...,80aa7bb2-adce-4a55-9711-912c407927a1,Non-Logged,2.216477
1,fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27...,d9e5f15d-b441-4d8b-bee4-462b106d3916,Non-Logged,2.429501
2,17f1083e6079b0f28f7820a6803583d1c1b405c0718b11...,e273dba4-136c-45fb-bdd6-0cc57b13aaf0,Non-Logged,1.794861


In [9]:
df_news = pd.read_csv(config["DF_ITEMS_FEATURE"])
df_news.drop(columns=["Unnamed: 0"],inplace=True)
df_news.head(3)

,page,age_exp_normalized,ageCategories
0,7371a9b5-5824-4c57-8704-00a74feebe79,0.151439,very-old
1,7a5ea08f-4583-49e2-ba52-a71999443f7b,0.140788,very-old
2,6afc8bbb-4f36-43d5-8a44-a2917df5621a,0.122610,very-old


In [10]:
df_valid_new = df_valid.drop(columns="timestampHistory")

from utils.custom_treat_data_funcs import transform_text_to_list, explode_df_columns

# df_valid_new["history"] = df_valid_new["history"].apply(str)
df_valid_new["history"] = df_valid_new["history"].apply(transform_text_to_list)
# df_valid_new.head(3)
df_valid_new = df_valid_new.explode("history", ignore_index=True)
df_valid_new.head(3)

,userId,userType,history
0,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,be89a7da-d9fa-49d4-9fdc-388c27a15bc8
1,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,01c59ff6-fb82-4258-918f-2910cb2d4c52
2,d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdee...,Logged,77901133-aee7-4f7b-afc0-652231d76fe9


In [11]:
df_valid_new.loc[0,"history"]

'be89a7da-d9fa-49d4-9fdc-388c27a15bc8'

In [12]:
df_valid_2 = pd.concat([df_ratings.loc[:10000], df_valid_new])

In [13]:
duplicates_v2 = df_valid_2[df_valid_2.duplicated(subset=['userId','history'], keep='first')]
duplicates_v2

,userId,history,userType,score
2855,477ca27ceb6450bab563b788e9e729e87a69067a20fcbb...,500df15c-57a0-4c2c-80e0-2d304c0b7477,Logged,<NA>
11575,210d9e1aa0c5b8dae98583c764d3d3ab859693f006c85e...,3c787cdc-99f7-4001-94ac-4edb2e971b94,Logged,<NA>
12474,320221eec958a51ac2baf1d3e53bb35ec2076b778bfb51...,35b32932-32d2-453d-b54c-125f346a380d,Logged,<NA>
12475,320221eec958a51ac2baf1d3e53bb35ec2076b778bfb51...,41f0d7df-cfe5-46de-ae95-a1c19e73b6d9,Logged,<NA>


In [14]:
df_valid_v2_cleaned = df_valid_2.drop_duplicates(subset=['userId','history'], keep="first")
df_valid_v2_cleaned

,userId,history,userType,score
0,fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27...,80aa7bb2-adce-4a55-9711-912c407927a1,Non-Logged,2.216477
1,fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27...,d9e5f15d-b441-4d8b-bee4-462b106d3916,Non-Logged,2.429501
2,17f1083e6079b0f28f7820a6803583d1c1b405c0718b11...,e273dba4-136c-45fb-bdd6-0cc57b13aaf0,Non-Logged,1.794861
3,528a8d7a2af73101da8d6709c1ec875b449a5a58749a99...,a0562805-c7d1-4ffd-b622-87c50ae006f4,Non-Logged,1.68271
4,2dd18b58a634a4e77181a202cf152df6169dfb3e4230ef...,233f8238-2ce0-470f-a9d5-0e0ac530382a,Non-Logged,2.266852
...,...,...,...,...
19036,eb82748abb8faabca586f788a10887170105475b16e575...,2277cab9-798b-4b0e-8f7c-cb2862ad3e4c,Logged,<NA>
19037,432bd2d2f562efcc59a3943b943b8783788c3526801761...,35e20551-659a-415f-95fe-7acbb89b69f6,Logged,<NA>
19038,d80b12031cb45593376c1c1776f52091ec24f3b09b671e...,5f05a0e2-5e92-47e3-b8c2-8d15ee772db3,Logged,<NA>
19039,1146778da61716481bd5bcfd985c948c52671d3d7a47d7...,f52361e4-9206-49c4-8117-2451c0b0c6f1,Logged,<NA>


In [15]:
df_valid_exploded = pd.concat([df_ratings, df_valid_new])
df_valid_exploded

,userId,history,userType,score
0,fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27...,80aa7bb2-adce-4a55-9711-912c407927a1,Non-Logged,2.216477
1,fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27...,d9e5f15d-b441-4d8b-bee4-462b106d3916,Non-Logged,2.429501
2,17f1083e6079b0f28f7820a6803583d1c1b405c0718b11...,e273dba4-136c-45fb-bdd6-0cc57b13aaf0,Non-Logged,1.794861
3,528a8d7a2af73101da8d6709c1ec875b449a5a58749a99...,a0562805-c7d1-4ffd-b622-87c50ae006f4,Non-Logged,1.68271
4,2dd18b58a634a4e77181a202cf152df6169dfb3e4230ef...,233f8238-2ce0-470f-a9d5-0e0ac530382a,Non-Logged,2.266852
...,...,...,...,...
19036,eb82748abb8faabca586f788a10887170105475b16e575...,2277cab9-798b-4b0e-8f7c-cb2862ad3e4c,Logged,<NA>
19037,432bd2d2f562efcc59a3943b943b8783788c3526801761...,35e20551-659a-415f-95fe-7acbb89b69f6,Logged,<NA>
19038,d80b12031cb45593376c1c1776f52091ec24f3b09b671e...,5f05a0e2-5e92-47e3-b8c2-8d15ee772db3,Logged,<NA>
19039,1146778da61716481bd5bcfd985c948c52671d3d7a47d7...,f52361e4-9206-49c4-8117-2451c0b0c6f1,Logged,<NA>


In [16]:
# duplicates = df_valid_exploded[df_valid_exploded.duplicated(subset=['userId','history'], keep=False)]
# duplicates = duplicates[duplicates["score"].isna()]
duplicates = df_valid_exploded[df_valid_exploded.duplicated(subset=['userId','history'], keep='first')]
duplicates
# duplicates = df_valid_exploded[df_valid_exploded.duplicated(subset=['history'], keep=False)]
# duplicates[duplicates["history"]=='253339a1-92b6-44d1-8fa2-59236c5251b1']
# duplicates[duplicates["userId"]=='f0e758359a184c99912e1ad5fc912b92b5e7b63c7a6002']

,userId,history,userType,score
75,9107ae20aa08ea8e46e27bb83801c8a77696a7499a2b6e...,c1c1e207-3a8c-4074-8973-56bdfed265b5,Logged,<NA>
99,e8f9ad90314928d72e21b7994c200cf01148d7f1779845...,b2a21491-7576-4be0-a520-97ce9470efeb,Logged,<NA>
501,ef38a8fb1bba38d09ec470e0e161c92c8b49d18cb1b1f8...,498f00e5-8860-4365-a03c-dca9fd298b80,Logged,<NA>
504,ef38a8fb1bba38d09ec470e0e161c92c8b49d18cb1b1f8...,31e64f8d-ca41-4831-8673-98c707014658,Logged,<NA>
505,ef38a8fb1bba38d09ec470e0e161c92c8b49d18cb1b1f8...,6bb54e2d-60ce-4ccb-b186-2d5bd093dc5c,Logged,<NA>
...,...,...,...,...
18431,6a155fd6955bbd3e9004bbd05ac01d26ebb7290cd1794b...,aeab0e46-f1e4-41e9-821b-571255c41f69,Logged,<NA>
18573,0bd79a0fbcc2842c8b245bacab378b7960838b79ace4ec...,9f8b2e17-f2de-44a8-8ddb-d2d2ea0c1469,Logged,<NA>
18645,cac745ed18f60476cd6226bf31c157b12ca22f5b97b82e...,0dd89516-0ca8-4884-90c0-a13b2f69ba37,Logged,<NA>
18720,9e02d2b143e498bf58b39344ccf54a472fc01b2ec5a848...,afd0d458-4efe-495f-92a2-ba9fcfc969ec,Logged,<NA>


In [17]:
df_valid_cleaned = df_valid_exploded.drop_duplicates(subset=['userId','history'], keep="first")
df_valid_cleaned

,userId,history,userType,score
0,fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27...,80aa7bb2-adce-4a55-9711-912c407927a1,Non-Logged,2.216477
1,fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27...,d9e5f15d-b441-4d8b-bee4-462b106d3916,Non-Logged,2.429501
2,17f1083e6079b0f28f7820a6803583d1c1b405c0718b11...,e273dba4-136c-45fb-bdd6-0cc57b13aaf0,Non-Logged,1.794861
3,528a8d7a2af73101da8d6709c1ec875b449a5a58749a99...,a0562805-c7d1-4ffd-b622-87c50ae006f4,Non-Logged,1.68271
4,2dd18b58a634a4e77181a202cf152df6169dfb3e4230ef...,233f8238-2ce0-470f-a9d5-0e0ac530382a,Non-Logged,2.266852
...,...,...,...,...
19036,eb82748abb8faabca586f788a10887170105475b16e575...,2277cab9-798b-4b0e-8f7c-cb2862ad3e4c,Logged,<NA>
19037,432bd2d2f562efcc59a3943b943b8783788c3526801761...,35e20551-659a-415f-95fe-7acbb89b69f6,Logged,<NA>
19038,d80b12031cb45593376c1c1776f52091ec24f3b09b671e...,5f05a0e2-5e92-47e3-b8c2-8d15ee772db3,Logged,<NA>
19039,1146778da61716481bd5bcfd985c948c52671d3d7a47d7...,f52361e4-9206-49c4-8117-2451c0b0c6f1,Logged,<NA>


In [18]:
df_valid_cleaned["score"].fillna(0,inplace=True)
df_valid_cleaned

/tmp/ipykernel_1561099/1963415375.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_valid_cleaned["score"].fillna(0,inplace=True)
/tmp/ipykernel_1561099/1963415375.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valid_cleaned["score"].fillna(0,inplace=True)


,userId,history,userType,score
0,fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27...,80aa7bb2-adce-4a55-9711-912c407927a1,Non-Logged,2.216477
1,fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27...,d9e5f15d-b441-4d8b-bee4-462b106d3916,Non-Logged,2.429501
2,17f1083e6079b0f28f7820a6803583d1c1b405c0718b11...,e273dba4-136c-45fb-bdd6-0cc57b13aaf0,Non-Logged,1.794861
3,528a8d7a2af73101da8d6709c1ec875b449a5a58749a99...,a0562805-c7d1-4ffd-b622-87c50ae006f4,Non-Logged,1.68271
4,2dd18b58a634a4e77181a202cf152df6169dfb3e4230ef...,233f8238-2ce0-470f-a9d5-0e0ac530382a,Non-Logged,2.266852
...,...,...,...,...
19036,eb82748abb8faabca586f788a10887170105475b16e575...,2277cab9-798b-4b0e-8f7c-cb2862ad3e4c,Logged,0.0
19037,432bd2d2f562efcc59a3943b943b8783788c3526801761...,35e20551-659a-415f-95fe-7acbb89b69f6,Logged,0.0
19038,d80b12031cb45593376c1c1776f52091ec24f3b09b671e...,5f05a0e2-5e92-47e3-b8c2-8d15ee772db3,Logged,0.0
19039,1146778da61716481bd5bcfd985c948c52671d3d7a47d7...,f52361e4-9206-49c4-8117-2451c0b0c6f1,Logged,0.0


In [19]:
dataset_valid = Dataset()

# Get unique values for users, items, and user features
unique_users_valid = df_valid_cleaned["userId"].unique()
unique_items_valid = df_valid_cleaned["history"].unique()
unique_user_features_valid = df_valid_cleaned["userType"].unique().tolist()

# Fit dataset with users, items, and user feature names
dataset_valid.fit_partial(
    users=unique_users_valid,
    items=unique_items_valid,
    user_features=unique_user_features_valid  # Register user features
)

In [20]:
(interactions_valid, weights_valid) = dataset_valid.build_interactions([
    (row.userId, row.history, row.score) 
    for _, row in df_valid_cleaned.iterrows()
])

In [21]:
print(unique_users_valid.shape)
print(unique_items_valid.shape)
print(len(unique_user_features_valid))

(51103,)
(66236,)
2


In [22]:
interactions_valid.shape

(51103, 66236)

In [23]:
user_features_list_valid = [
    (row.userId, [row.userType])  
    for _, row in df_valid.iterrows()
]

user_features_valid = dataset_valid.build_user_features(user_features_list_valid)

In [24]:
# print(user_features_valid)

In [25]:
import pickle

loaded_model = pickle.load(open('artifacts/lightfm_model.pkl', 'rb'))
loaded_user_id_map = pickle.load(open('artifacts/user_id_map.pkl', 'rb'))
loaded_item_id_map_reverse = pickle.load(open('artifacts/item_id_map_reverse.pkl', 'rb'))
loaded_user_feature_map = pickle.load(open('artifacts/user_feature_map.pkl', 'rb'))

In [26]:
loaded_model.fit_partial(interactions=interactions_valid, sample_weight=weights_valid, user_features=user_features_valid,
                         epochs=NO_EPOCHS, num_threads=NUM_THREADS)
# loaded_model.fit_partial(interactions=interactions_valid, sample_weight=weights_valid,
#                          epochs=NO_EPOCHS, num_threads=NUM_THREADS)

In [27]:
user_id_map, user_feature_map, item_id_map, item_feature_map = dataset_valid.mapping()
item_id_map_reverse = {v: k for k, v in item_id_map.items()}

loaded_user_id_map = user_id_map
loaded_item_id_map_reverse = item_id_map_reverse
loaded_user_feature_map = user_feature_map

In [28]:
# _, n_items = interactions.shape # no of users * no of items
_, n_items_valid = interactions_valid.shape # no of users * no of items

# n_items = NUM_ITEMS_TRAIN
n_items = n_items_valid

### Make predictions to known and unknowm on same recommendation function with pkls

In [29]:
def format_newuser_input(user_feature_map, user_feature_list):
  normalised_val = 1.0 
  target_indices = []
  for feature in user_feature_list:
    try:
        target_indices.append(user_feature_map[feature])
    except KeyError:
        # print("new user feature encountered '{}'".format(feature))
        pass
  #print("target indices: {}".format(target_indices))
  new_user_features = np.zeros(len(user_feature_map.keys()))
  for i in target_indices:
    new_user_features[i] = normalised_val
  new_user_features = sparse.csr_matrix(new_user_features)
  return(new_user_features)

In [30]:
def sample_recommendation_by_title(user_hash,df_news,user_feature_list,item_id_map_reverse,user_feature_map,user_id_map,model):
    try:
        user_x = user_id_map[user_hash]
        scores = model.predict(user_x, np.arange(n_items)) # means predict for all
    except:
        new_user_features = format_newuser_input(user_feature_map, user_feature_list)
        scores = model.predict(0, np.arange(n_items), user_features=new_user_features)
    
    top_5_indices = np.argsort(-scores)[:10]  # Sort scores in descending order and take the top 5
    top_5_items = [item_id_map_reverse[i] for i in top_5_indices]

    print("Top 5 recommended items:")

    for x in top_5_items:
        row = df_news[df_news["page"] == x]
        print("        %s" % row["title"].values[0])


In [31]:
def get_recommended_history_list(user_hash,user_feature_list,item_id_map_reverse,user_feature_map,user_id_map,model):
    """
    This function verifies if the users is known or new, and makes recommendations depending on this verification.
    The top 5 recommendations from the list are returned.
    """
    try:
        user_x = user_id_map[user_hash]
        scores = model.predict(user_x, np.arange(n_items)) # means predict for all
    except:
        new_user_features = format_newuser_input(user_feature_map, user_feature_list)
        # scores = model.predict(0, np.arange(n_items))
        scores = model.predict(0, np.arange(n_items), user_features=new_user_features)
    
    top_5_indices = np.argsort(-scores)[:20]  # Sort scores in descending order and take the top 5
    top_5_items = [item_id_map_reverse[i] for i in top_5_indices]

    return top_5_items

In [32]:
def df_implode(df:pd.DataFrame,group_by_columns:list, implode_column:str):
    df = (df.groupby(group_by_columns)\
      .agg({implode_column: lambda x: ",".join(x)})\
      .reset_index())
    return df

df_valid_2 = df_valid_2[["userId","userType","history"]]

df_valid_imploded = df_implode(df_valid_2, ["userId","userType"], "history")
df_valid_imploded["history"] = df_valid_imploded["history"].apply(transform_text_to_list)
df_valid_imploded

,userId,userType,history
0,000a55dd57af328ad54f083f2a5fef9bf4ea68e71a951f...,Logged,[esid:conteudo_editorial_g1#materia#http://esp...
1,000b0468cadd863bd27f22328cf119a98174fa3c5c1692...,Logged,"[ea6be706-8c16-4564-ab4d-b0ddbc9376e9, a546d8d..."
2,000f8ffb33211a68b4008de5c902b5fab7538142ee2fd0...,Logged,[0975ed21-0401-40fb-b161-bfc84fa4bfa4]
3,001055fb8d4aca0ce7eb4e3d33fee234c8e0c5b1410718...,Logged,"[eb23272d-8e6c-479d-b972-eabeb5f6f3dd, 17c1632..."
4,0010cbb9e4fd6b426ac740d39063c6822e854f3d45893e...,Logged,"[0a875e6e-56f6-40a8-9b8d-1a3bdbd63c83, 5de99d9..."
...,...,...,...
11039,fff46e72c87ef6d8e149b0a60f3346a84256b2d30c04bc...,Logged,[31e64f8d-ca41-4831-8673-98c707014658]
11040,fff4dfed09ba5f2babf7fa198875634777b1770939226b...,Logged,"[e0d87b73-9f23-4a82-8a56-a0c416f74a9e, 82c2e23..."
11041,fff72f1e3d25c5027177850a217129bf088902d4ee1103...,Logged,[f8b246a9-bc8f-4186-ad33-70d7c266b89c]
11042,fff9dce3d8c37244e389776b734df2e663bc7c9c707b38...,Logged,"[ca380869-b9c2-4407-b715-df53f9cdf817, 48eddf2..."


In [33]:
# Testing for just one user

user_feature_list = [df_valid["userType"].iloc[0]]
user_hash = df_valid["userId"].iloc[0]
validation_history_hashes = df_valid["history"].iloc[0]

recommeded_histories = get_recommended_history_list(user_hash,user_feature_list,loaded_item_id_map_reverse,loaded_user_feature_map,loaded_user_id_map,loaded_model)
print(recommeded_histories)
print(validation_history_hashes)

['d2593c3d-2347-40d9-948c-b6065e8459a9', 'f6b5d170-48b9-4f8e-88d4-c84b6668f3bd', '1f32787b-de2b-49be-8c20-ddaeae34cc22', '6a83890a-d9e9-4f6b-a6c6-90d031785bbf', 'f0a78e58-ec7e-494c-9462-fbd6446a9a89', '4c63d7cd-4902-4ffb-9b94-578b1b2151f0', '855d20b7-53f2-4678-a10f-55402d085018', 'bf257382-74fb-4392-ad6a-143240e39f81', '1c27cf97-b20c-4e40-b1f1-288b721517b3', 'a36c98b5-f159-48f8-9f5a-1fc6ea9956c8', '4e9c2825-ff13-41ca-8e91-edd848060d19', '882e7c95-935a-4eab-9ece-f85f5f7d0f4e', '15281e10-e6bc-48bc-9b1b-94402f83699b', '8c246d2b-81bd-4c1f-b563-2c905675f984', '89fa73f0-4341-4de4-bb2a-e429ef96bd43', '29b6b142-4173-4ec4-832f-7d0a32255c10', 'e384ec29-136e-4241-9321-49b367b8cbd5', '7b056bf6-c232-46bd-8903-59145ff7ce46', '4700f517-5c5d-483c-81b7-d77aca04991c', '458bf0ec-efb4-4bfd-9446-c80295e6aa87']
['be89a7da-d9fa-49d4-9fdc-388c27a15bc8'
 '01c59ff6-fb82-4258-918f-2910cb2d4c52']


In [34]:
def count_valid_recommendations(validation_history_hashes, recommeded_histories):
    """
    This function receives 
    * The `validation_history_hashes` (which is a list of histories contained in the "validacao.csv")
    * The `recommeded_histories` (which are the recommended histories/items by the model)
    Then, it verifies how many recommended items match with the validation histories.
    """
    count_valids = 0
    for valid_hist in validation_history_hashes:
        if valid_hist in recommeded_histories:
            count_valids = count_valids+1
    return count_valids

In [35]:
df_valid["recommended_hists"] = "" # creates a column to receive the list of recommended histories
df_valid["matched_recommendations"] = 0 # creates a column to receive the quantity of matches between recommendations and validation items
df_valid["historySize"] = df_valid["history"].apply(lambda x : len(x)) # calculate the quantity of histories for each user

In [36]:
df_valid.head(2)

,userId,userType,history,timestampHistory,recommended_hists,matched_recommendations,historySize
0,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,['be89a7da-d9fa-49d4-9fdc-388c27a15bc8' '01c5...,[1660533136590 1660672113513],,0,80
1,d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdee...,Logged,['77901133-aee7-4f7b-afc0-652231d76fe9'],[1660556860253],,0,40


In [37]:
import multiprocessing

def parallelize_dataframe(df, func):
    num_cores = multiprocessing.cpu_count()-1  #leave one free to not freeze machine
    num_partitions = num_cores #number of partitions to split dataframe
    df_split = np.array_split(df, num_partitions)
    pool = multiprocessing.Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [38]:
df_valid_imploded["recommended_hists"] = ""
df_valid_imploded["matched_recommendations"] = 0

In [39]:
df_valid["recommended_hists"] = ""
df_valid["matched_recommendations"] = 0

In [40]:
# def validate_recommendations(df_valid):
for index, row in df_valid.iterrows():
    """
    For each user, get the top 5 recommendations from the model, and also count how many of them are contained within the validation set.
    Stores the results on the columns `recommended_hists` `matched_recommendations`.
    """
    user_feature_list = [row["userType"]]
    user_hash = row["userId"]
    validation_history_hashes = row["history"]

    recommended_hist = get_recommended_history_list(user_hash,user_feature_list,loaded_item_id_map_reverse,loaded_user_feature_map,loaded_user_id_map,loaded_model)
    num_valid_recommendations = count_valid_recommendations(validation_history_hashes, recommended_hist)

    # df_valid.loc[index, 'recommended_hists'] = recommended_hist
    df_valid.loc[index, 'matched_recommendations'] = num_valid_recommendations
        

In [41]:
# def validate_recommendations(df_valid):
for index, row in df_valid_imploded.iterrows():
    """
    For each user, get the top 5 recommendations from the model, and also count how many of them are contained within the validation set.
    Stores the results on the columns `recommended_hists` `matched_recommendations`.
    """
    user_feature_list = [row["userType"]]
    user_hash = row["userId"]
    validation_history_hashes = row["history"]

    recommended_hist = get_recommended_history_list(user_hash,user_feature_list,loaded_item_id_map_reverse,loaded_user_feature_map,loaded_user_id_map,loaded_model)
    num_valid_recommendations = count_valid_recommendations(validation_history_hashes, recommended_hist)

    # df_valid_imploded.loc[index, 'recommended_hists'] = recommended_hist
    df_valid_imploded.loc[index, 'matched_recommendations'] = num_valid_recommendations

In [42]:
df_valid_imploded.describe()

,matched_recommendations
count,11044.000000
mean,0.027526
std,0.252782
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,10.000000


In [43]:
df_valid.describe()

,matched_recommendations,historySize
count,10000.0,10000.000000
mean,0.0,81.639700
std,0.0,61.423562
min,0.0,40.000000
25%,0.0,40.000000
50%,0.0,40.000000
75%,0.0,120.000000
max,0.0,765.000000
